# Preprocessing 
### Input Folder structure:

```
notebook
├── boreal_agb_density_ICESat2_tiles_shp
|       - Boreal_AGB_Density_ICESat2_tiles.dbf
|       - Boreal_AGB_Density_ICESat2_tiles.prj
|       - Boreal_AGB_Density_ICESat2_tiles.sbn
|       - Boreal_AGB_Density_ICESat2_tiles.sbx
|       - Boreal_AGB_Density_ICESat2_tiles.shp
|       - Boreal_AGB_Density_ICESat2_tiles.shx
├── data
|   - download_icesat.sh
├── S2_tiles_Siberia_polybox
|   - S2_tiles_Siberia_above_GEDI.geojson
|   - S2_tiles_Siberia_all.geojson
|   - S2_tiles_Siberia_within_GEDI.geojson
- preprocessing.ipynb
```

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import csv
import os
import rasterio as rs
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask

##### Finding the corresponding IceSat-2 tiles for every Sentinel-2 tile

In [24]:
#reading in Sentinel 2 shapefile
s2_df = gpd.read_file('S2_tiles_Siberia_polybox/S2_tiles_Siberia_all.geojson', driver='GeoJSON', index = False) 
# print(s2_df.head())
#reading in ICESat2 shapefile
icesat_df = gpd.read_file('boreal_agb_density_ICESat2_tiles_shp/Boreal_AGB_Density_ICESat2_tiles.shp', driver='ESRI Shapefile', index = False)
# print(icesat_df.head())
        
overlapping_tiles = pd.DataFrame()
#iterating over each Sentinel 2 tile and finding the ICESat2 tiles that overlap with it
for _, s2_tile in s2_df.iterrows():
    s2_geometry = s2_tile.geometry
    overlapping_icesat_tiles = icesat_df[icesat_df.geometry.intersects(s2_geometry)].copy()

    # adding the name of the S2 tile to the ICESat2 tiles that overlap with it
    if not overlapping_icesat_tiles.empty:
        overlapping_icesat_tiles.loc[:, 'S2 Tile Name'] = s2_tile.Name
        # overlapping_icesat_tiles.loc[:, 'S2 Tile Geometry'] = s2_geometry
    
    overlapping_tiles = pd.concat([overlapping_tiles, overlapping_icesat_tiles])

# overlapping_tiles_df = gpd.GeoDataFrame(overlapping_tiles, columns=['S2 Tile Name', 'Overlapping ICESat Tiles'])
overlapping_tiles.to_csv('overlapping_tiles.csv', index=False)

##### Preparing to download, create a 'links.txt' file which contains the download links of the relevant IceSat-2 granules

In [27]:
csv_file = 'overlapping_tiles.csv'

# Get the column index of "GeoTiff"
geo_tiff_column_index = overlapping_tiles.columns.get_loc("GeoTIFF")

filenames = []

with open(csv_file, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row

    for row in csv_reader:
        if row[geo_tiff_column_index] not in filenames:
            filenames.append(row[geo_tiff_column_index])

# print(filenames)

with open('links.txt', 'w') as file:
    for filename in filenames:
        file.write('https://data.ornldaac.earthdata.nasa.gov/protected/above/Boreal_AGB_Density_ICESat2/data/' + filename + '\n')

## Run 'download_icesat.sh' 
The script should be located in the '/data' folder. Replace 'username' and 'password' with your ***correct*** earthdata credentials

Open a terminal in the '/data' folder and run:

\>\> chmod 777 download_icesat.sh

\>\> ./download_icesat.sh

---

#### Mosaic the tiles together

This will create a "/merged_mosaic" folder where for every Sentinel-2 tile we save the overlapping IceSat-2 tiles mosaiced together as [S2-tilename]_mosaic.tif These mosaiced tiles are still in the IceSat-2 crs

In [6]:
# Specify the path to the CSV file
if 'csv_file' not in globals():
    csv_file = 'overlapping_tiles.csv'
if 'df' not in globals():
    # Read the CSV file as a DataFrame
    df = pd.read_csv(csv_file)
if 'unique_tiles' not in globals():
    # Get the unique values in the "S2 Tile Name" column
    unique_tiles = df['S2 Tile Name'].unique()
if 's2_df' not in globals():
    s2_df = gpd.read_file('S2_tiles_Siberia_polybox/S2_tiles_Siberia_all.geojson', driver='GeoJSON', index = False)


# Iterate over the unique tiles
for tile in unique_tiles:
    print(f'Processing tile {tile}...')
    entry = s2_df[s2_df['Name']==tile]
    icesat_tiles = []
    for index, row in df[df['S2 Tile Name']==tile].iterrows():
        icesat_tiles.append(row['GeoTIFF'])
    
    to_merge = []
    # Construct the path to the corresponding tif file
    for icesat_tile in icesat_tiles:
        temp_tiff = rs.open(f'data/{icesat_tile}')
        to_merge.append(temp_tiff)
    
    assert len(to_merge)!=0, "No corresponding ICESat-2 tiles found"
    # Check if the CRS of all the to_merge are the same
    crs_check = all(to_merge[0].crs == t.crs for t in to_merge)
    if not crs_check:
        raise ValueError("CRS of the to_merge tiles are not the same")
  
    # Merge the tiles
    mosaic, mosaic_trans = merge(to_merge)
    mosaic_crs = to_merge[0].crs

    height, width = mosaic.shape[1], mosaic.shape[2]
    output_meta = to_merge[0].meta.copy()
    output_meta.update({"driver": "GTiff",
                        "height": height,
                        "width": width,
                        "transform": mosaic_trans,
                        "count": mosaic.shape[0]})
    output_path = f'merged_mosaic/{tile}_mosaic.tif'

    # Specify the path to the merged_mosaic folder
    merged_mosaic_folder = 'merged_mosaic'

    # Check if the merged_mosaic folder exists
    if not os.path.exists(merged_mosaic_folder):
        # Create the merged_mosaic folder
        os.makedirs(merged_mosaic_folder)
    
    with rs.open(output_path, "w", **output_meta) as dest:
        dest.write(mosaic)


    # Close the tiles' files
    for src in to_merge : src.close()


Processing tile 50TPS...
Processing tile 50TPT...
Processing tile 50TQS...
Processing tile 50TQT...
Processing tile 50UQA...
Processing tile 50UQB...
Processing tile 51TVL...
Processing tile 51TVM...
Processing tile 51TVN...
Processing tile 51TWM...
Processing tile 51TWN...
Processing tile 51UUP...
Processing tile 51UUQ...
Processing tile 51UUT...
Processing tile 51UVP...
Processing tile 51UVQ...
Processing tile 51UVR...
Processing tile 51UVS...
Processing tile 51UVT...
Processing tile 51UWP...
Processing tile 51UWQ...
Processing tile 51UWR...
Processing tile 51UWS...
Processing tile 51UWT...
Processing tile 51UXQ...
Processing tile 51UXR...
Processing tile 51UXS...
Processing tile 51UXT...
Processing tile 51UYS...
Processing tile 51UYT...
Processing tile 52UCA...
Processing tile 52UDA...
Processing tile 52UDB...
Processing tile 52UDC...
Processing tile 52UEA...
Processing tile 52UEB...
Processing tile 52UEC...
Processing tile 52UEV...
Processing tile 52UFA...
Processing tile 52UFB...


#### Reprojecting the mosaics

We reproject the mosaics from the IceSat-2 crs to the Sentinel-2 crs and save them to the "/reprojected_mosaic" folder

In [11]:
# this is a helper function to get the CRS of the Sentinel-2 tile from its name
# it is used in both reprojecting and cropping the tiles
def get_CRS_from_S2_tilename(tname) :
    """
    Get the CRS of the Sentinel-2 tile from its name. The tiles are named as DDCCC (where D is a digit and C a character).
    MGRS tiles are in UTM projection, which means the CRS will be EPSG=326xx in the Northern Hemisphere, and 327xx in the
    Southern. The first character of the tile name gives you the hemisphere (C to M is South, N to X is North); and the
    two digits give you the UTM zone number.

    Args:
    - tname: str, name of the Sentinel-2 tile

    Returns:
    - rasterio.crs.CRS, the CRS of the Sentinel-2 tile
    """

    tile_code, hemisphere = tname[:2], tname[2]

    if 'C' <= hemisphere <= 'M':
        crs = f'EPSG:327{tile_code}'
    elif 'N' <= hemisphere <= 'X':
        crs = f'EPSG:326{tile_code}'
    else:
        raise ValueError(f'Invalid hemisphere code: {hemisphere}')
    
    return rs.crs.CRS.from_string(crs)

In [ ]:
# Open the original raster dataset
def custom_reproject(src_path, output_path, dst_crs):
    # Open the original raster dataset
    with rs.open(src_path) as src:
        
        # Calculate the transformation parameters and the width and height of the output
        # in the new CRS
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        
        # Copy the metadata from the source dataset
        kwargs = src.meta.copy()
        # Update the metadata with the new CRS, transformation, and dimensions
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Specify the path to the merged_mosaic folder
        reprojected_mosaic_folder = 'reprojected_mosaic'

        # Check if the merged_mosaic folder exists
        if not os.path.exists(reprojected_mosaic_folder):
            # Create the merged_mosaic folder
            os.makedirs(reprojected_mosaic_folder)

        # Open a new raster file for the reprojected data
        with rs.open(output_path, 'w', **kwargs) as dst:
            
            # Reproject each band in the raster dataset
            for i in range(1, src.count + 1):
                reproject(
                    # Source raster band
                    source=rs.band(src, i),
                    # Destination raster band
                    destination=rs.band(dst, i),
                    # Source transformation and CRS
                    src_transform=src.transform,
                    src_crs=src.crs,
                    # Destination transformation and CRS
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    # Resampling method
                    resampling=Resampling.nearest)


# Specify the path to the CSV file
if 'csv_file' not in globals():
    csv_file = 'overlapping_tiles.csv'
if 'df' not in globals():
    # Read the CSV file as a DataFrame
    df = pd.read_csv(csv_file)
if 'unique_tiles' not in globals():
    # Get the unique values in the "S2 Tile Name" column
    unique_tiles = df['S2 Tile Name'].unique()
if 's2_df' not in globals():
    s2_df = gpd.read_file('S2_tiles_Siberia_polybox/S2_tiles_Siberia_all.geojson', driver='GeoJSON', index = False)

    
# Iterate over the unique tiles
for tile in unique_tiles:
    print(f'Processing tile {tile}...')
    entry = s2_df[s2_df['Name']==tile]

    dest_crs = get_CRS_from_S2_tilename(tile)

    # Read in the (tile)_mosaic.tif file
    mosaic_file = f'merged_mosaic/{tile}_mosaic.tif'
    custom_reproject(mosaic_file, f'reprojected_mosaic/{tile}_reprojected_mosaic.tif', dest_crs)


#### Cropping the reprojected mosaics
We crop the reprojected mosaics to the corresponding Sentinel-2 tile and save them to the "/cropped_mosaic" folder.

In [ ]:
# Specify the path to the CSV file
if 'csv_file' not in globals():
    csv_file = 'overlapping_tiles.csv'
if 'df' not in globals():
    # Read the CSV file as a DataFrame
    df = pd.read_csv(csv_file)
if 'unique_tiles' not in globals():
    # Get the unique values in the "S2 Tile Name" column
    unique_tiles = df['S2 Tile Name'].unique()
if 's2_df' not in globals():
    s2_df = gpd.read_file('S2_tiles_Siberia_polybox/S2_tiles_Siberia_all.geojson', driver='GeoJSON', index = False)

# Specify the directory path
directory = 'reprojected_mosaic'

# Iterate over the unique tiles
for tile in unique_tiles:
    print(f'Processing tile {tile}...')
    # Get the file paths of all the files in the directory
    file_path = os.path.join(directory, tile + "_reprojected_mosaic.tif")
    entry = s2_df[s2_df['Name']==tile]

    
    #reproject the tiles to the same CRS
    dest_crs = get_CRS_from_S2_tilename(tile)
    # print("entry.crs:  " + str(entry.crs))
    reprojected_entry = entry.to_crs(dest_crs, inplace=False)
    # print(reprojected_entry.crs.to_epsg())

    # Specify the path to the merged_mosaic folder
    cropped_mosaic_folder = 'cropped_mosaic'

    # Check if the merged_mosaic folder exists
    if not os.path.exists(cropped_mosaic_folder):
        # Create the merged_mosaic folder
        os.makedirs(cropped_mosaic_folder)


    with rs.open(file_path) as dataset:
        # Access the data or perform any required operations
        data = dataset.read()
        # Do something with the data
        out_image, out_transform = mask(dataset, reprojected_entry.geometry, crop=True)
        out_meta = dataset.meta.copy()
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
        output_path = f'cropped_mosaic/{tile}_cropped_mosaic.tif'   
        with rs.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)


# Output
After running the above code we should now have the following folder structure:

```
notebook
├── boreal_agb_density_ICESat2_tiles_shp
├── cropped_mosaic
├── data
├── merged_mosaic
├── reprojected_mosaic
└── S2_tiles_Siberia_polybox
- links.txt
- overlapping_tiles.csv
- preprocessing.ipynb
```

As a last step we replace the nan values in the cropped mosaics with -9999 and save them into a folder "cropped_mosaic_no_nan"

In [6]:
import os
import numpy as np
import rasterio

# Define the paths to the input and output folders
input_folder = "cropped_mosaic"
output_folder = "cropped_mosaic_no_nan"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get the list of TIFF files in the input folder
tif_files = [file for file in os.listdir(input_folder) if file.endswith(".tif")]

# Iterate over each TIFF file
for tif_file in tif_files:
    print(f"Processing {tif_file}...")
    # Read the input TIFF file
    input_path = os.path.join(input_folder, tif_file)
    with rasterio.open(input_path) as src:
        # Read the raster data as a numpy array
        data1 = src.read(1)
        data2 = src.read(2)
        
        # Replace NaN values with -9999
        data1[np.isnan(data1)] = -9999
        data2[np.isnan(data2)] = -9999
        
        # Create the output TIFF file path
        output_path = os.path.join(output_folder, tif_file)
        
        # Copy the metadata from the input file
        profile = src.profile
        
        # Write the modified data to the output TIFF file
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(data1, 1)
            dst.write(data2, 2)


Processing 50TPS_cropped_mosaic.tif...
Processing 50TPT_cropped_mosaic.tif...
Processing 50TQS_cropped_mosaic.tif...
Processing 50TQT_cropped_mosaic.tif...
Processing 50UQA_cropped_mosaic.tif...
Processing 50UQB_cropped_mosaic.tif...
Processing 51TVL_cropped_mosaic.tif...
Processing 51TVM_cropped_mosaic.tif...
Processing 51TVN_cropped_mosaic.tif...
Processing 51TWM_cropped_mosaic.tif...
Processing 51TWN_cropped_mosaic.tif...
Processing 51UUP_cropped_mosaic.tif...
Processing 51UUQ_cropped_mosaic.tif...
Processing 51UUT_cropped_mosaic.tif...
Processing 51UVP_cropped_mosaic.tif...
Processing 51UVQ_cropped_mosaic.tif...
Processing 51UVR_cropped_mosaic.tif...
Processing 51UVS_cropped_mosaic.tif...
Processing 51UVT_cropped_mosaic.tif...
Processing 51UWP_cropped_mosaic.tif...
Processing 51UWQ_cropped_mosaic.tif...
Processing 51UWR_cropped_mosaic.tif...
Processing 51UWS_cropped_mosaic.tif...
Processing 51UWT_cropped_mosaic.tif...
Processing 51UXQ_cropped_mosaic.tif...
Processing 51UXR_cropped_